# Evaluating Strands Agent with Observability with LangFuse

## Overview
In this module, we'll explore how to implement observability for [Strands Agent](https://strandsagents.com/latest/) using [Langfuse](https://langfuse.com/). Strands Agent enables you to build an intelligent AWS assistant by connecting foundation models to AWS services and resources, here's the link to [Strands Agent SDK's GitHub repo](https://github.com/strands-agents). While powerful AI agents can solve complex problems, understanding their behavior and performance requires robust observability solutions.

Langfuse integration provides comprehensive visibility into your agent's operations, helping you monitor, debug, and optimize its performance. This module will guide you through implementing and leveraging Langfuse observability with Strands Agent to create more reliable, transparent, and effective AI solutions.

Here's what you will learn from this module:
1. Setup Lanfuse Observability 
2. Create a Strands Agent with [built-in tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/) and trace it with Lanfuse
3. Create a Strands Agent with [custom tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/python-tools/) and trace it with Lanfuse
4. Lastly create a Strands Agent with [MCP tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/mcp-tools/) and trace it with Lanfuse

## Agent Details
<div style="float: left; margin-right: 20px;">
    
|Feature             |Description                                              |
|--------------------|---------------------------------------------------------|
|Native tools used   |image_reader                                             |
|Custom tools created|aws_health_status_checker                                |
|MCP Tools           |aws_documentation_mcp_tool, aws_pricing_mcp_tool         |
|Agent Structure     |Single agent architecture                                |
|AWS services used   |Amazon Bedrock                                           |
|Integrations        |LangFuse for observability                               |

</div>



## Architecture

<div style="text-align:left">
    <img src="./image/architecture.png" width="75%" />
</div>

## Key Features
- Fetches Strands agent interaction traces from Langfuse. You can also save these traces offline and use them here without Langfuse.
- Evaluates conversations using specialized metrics for agents, tools
- Pushes evaluation scores back to Langfuse for a complete feedback loop
- Evaluate both single-turn (with context) and multi-turn conversations

## Setup and prerequisites

### Prerequisites
* Python 3.10+
* AWS account
* Anthropic Claude 4.0 on Amazon Bedrock
* LangFuse Key

Let's now install the requirement packages for our Strands Agent

In [ ]:
# Uncomment the following line to install dependencies if you are not using AWS workshop environment
# %pip install -q langfuse boto3  --upgrade

In [ ]:
!uv pip install --force-reinstall -U -r ./requirements.txt --quiet

Please make sure you have completed the prerequisites to setup the Langfuse project and API keys in the .env file to connect to self-hosted or cloud Langfuse environment.

1. Navigate to the directory `genai-ml-platform-examples/integration/genaiops-langfuse-on-aws/` within your workshop environment.

2. Locate the file named `.env.example` and create a copy of this file in the same directory, renaming the copy to `.env`.

3. Open the `.env` file in your editor and prepare to add your actual Langfuse credentials. You will need three values from your Langfuse project settings under the API Keys section.

The completed configuration in `.env` should follow this format:

```
LANGFUSE_PUBLIC_KEY=pk-lf-your-actual-public-key
LANGFUSE_SECRET_KEY=sk-lf-your-actual-secret-key
LANGFUSE_HOST=xxx
```

Save the file after adding your actual credential values. The notebook will load these environment variables automatically when executing the Langfuse integration exercises in agents two and three.

In [ ]:
# If you completed the .env setup above, skip this cell.
# Otherwise, uncomment and set your Langfuse credentials below:

# import os
# os.environ["LANGFUSE_SECRET_KEY"] = "sk-lf-..."  # Your Langfuse project secret key
# os.environ["LANGFUSE_PUBLIC_KEY"] = "pk-lf-..."  # Your Langfuse project public key
# os.environ["LANGFUSE_HOST"] = "xxx"  # Your Langfuse host URL

### Importing dependency packages

Now let's import the dependency packages

In [ ]:
import base64
import json
import os

from dotenv import load_dotenv

# Import MCP related modules
from mcp import StdioServerParameters, stdio_client
from strands import Agent, tool

# Import Strands telemetry for OpenTelemetry integration
from strands.telemetry import StrandsTelemetry
from strands.tools.mcp import MCPClient
from strands_tools import image_reader


load_dotenv("../.env")

## Setting Strands Agents to emit LangFuse traces
#### The first step here is to set Strands Agents to emit traces to LangFuse

#### Configure the telemetry(Creates new tracer provider and sets it as global)

In [ ]:
# Get keys for your project from the project settings page: https://cloud.langfuse.com
langfuse_public_key = os.environ.get("LANGFUSE_PUBLIC_KEY")
langfuse_secret_key = os.environ.get("LANGFUSE_SECRET_KEY")
langfuse_host = os.environ.get("LANGFUSE_HOST", "https://cloud.langfuse.com")

if langfuse_public_key and langfuse_secret_key:
    LANGFUSE_AUTH = base64.b64encode(f"{langfuse_public_key}:{langfuse_secret_key}".encode()).decode()

    os.environ["OTEL_EXPORTER_OTLP_ENDPOINT"] = langfuse_host + "/api/public/otel"
    os.environ["OTEL_EXPORTER_OTLP_HEADERS"] = f"Authorization=Basic {LANGFUSE_AUTH}"
    print("Langfuse observability configured.\n")

else:
    print("Warning: Langfuse API keys not found. Observability will not be enabled.")

In [ ]:
try:
    strands_telemetry = StrandsTelemetry().setup_otlp_exporter()
    print("OpenTelemetry exporter configured for Langfuse.")
except ImportError:
    print("Warning: OpenTelemetry exporter not available. Install with 'pip install opentelemetry-exporter-otlp'.")
    strands_telemetry = None

## Build your first strands agent
### Using the built in tool
Tools are the primary mechanism for extending agent capabilities, enabling them to perform actions beyond simple text generation. Tools allow agents to interact with external systems, access data, and manipulate their environment.
Strands offers built-in example tools to get started quickly experimenting with agents and tools during development. For more information, see Example [Built-in Tools](https://strandsagents.com/latest/documentation/docs/user-guide/concepts/tools/example-tools-package/).

In [ ]:
# Customer query
query = """
What is in the picture of image/aws-architecture.png in the current folder path
"""

# Building the strands agent with built-in tool image_read and including Lanfuse for tracing
agent = Agent(
    tools=[image_reader],
    model="global.anthropic.claude-sonnet-4-5-20250929-v1:0",  # Using Claude 4.0 Sonnet via Bedrock
    name="strands-agent-built-in-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "example-user@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Built-In-Tool",
        ],
        "metadata": {"environment": "development", "version": "1.0.0", "query_type": "storage_recommendation"},
    },
)

try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

### Now go to your Lanfuse console, you should be able to see a new trace with the name "strands_agent_built-in-tool-example" appear. Have a look at what's in the trace.

<div style="text-align:left">
    <img src="./image/trace1.jpg" width="15%" />
    <img src="./image/trace2.png" width="75%" />
</div>

## Strands Agents with custom tools
Define tools by creating Python modules that contain a tool specification and a matching function. This approach gives you more control over the tool's definition and is useful for dependency-free implementations of tools.

In [ ]:
import xml.etree.ElementTree as ET

import requests


# Using the @tool python decorator, creating a custom tool that uses the AWS Health Dashboard RSS feed URL to check AWS service status
@tool
def aws_health_status_checker(region: str = "us-west-2", service_name: str | None = None) -> dict:
    """
    Check the current operational status of AWS services in a specific region using the public RSS feed.

    Args:
        region: AWS region to check (e.g., us-east-1, us-west-2)
        service_name: Optional specific service to check (e.g., ec2, s3, lambda)
                     If not provided, will return status for all services

    Returns:
        Dictionary containing service health information
    """
    try:
        # AWS Health Dashboard RSS feed URL
        rss_url = "https://status.aws.amazon.com/rss/all.rss"

        # Get the RSS feed
        response = requests.get(rss_url, timeout=10)
        response.raise_for_status()

        # Parse the XML
        root = ET.fromstring(response.content)

        # Find all items (service events)
        items = root.findall(".//item")

        # Filter events by region and service
        events = []
        for item in items:
            title = item.find("title").text if item.find("title") is not None else ""
            description = item.find("description").text if item.find("description") is not None else ""
            pub_date = item.find("pubDate").text if item.find("pubDate") is not None else ""
            link = item.find("link").text if item.find("link") is not None else ""

            # Check if this event is for the requested region and service
            if region.lower() in title.lower() and (service_name is None or service_name.lower() in title.lower()):
                events.append(
                    {"title": title, "description": description, "published_date": pub_date, "link": link}
                )

        if not events:
            return {
                "status": "healthy",
                "message": f"All services in {region} appear to be operating normally"
                if not service_name
                else f"{service_name} in {region} appears to be operating normally",
                "events": [],
            }

        return {
            "status": "service_disruption",
            "message": f"Service disruptions detected in {region}",
            "events": events,
        }

    except Exception as e:
        return {"status": "error", "message": f"Error checking AWS service status: {str(e)}", "events": []}

#### Create a Strands Agent with this custom tool

In [ ]:
# Customer query
query = """
Check AWS service status in the us-west-2 region.
"""

# Building the strands agent with the custom tool "aws_health_status_checker" defined above
agent = Agent(
    system_prompt="You are an AWS service checking agent, use the aws_service_status_checker to tell the user if the service status that they ask",
    tools=[aws_health_status_checker],
    model="global.anthropic.claude-sonnet-4-5-20250929-v1:0",  # Using Claude 4.5 Sonnet via Bedrock
    name="strands-agent-custom-tool-example",
    trace_attributes={
        "session.id": "aws-mcp-agent-demo-session",
        "user.id": "example-user@example.com",
        "langfuse.tags": [
            "AWS-Strands-Agent",
            "Custom-Tool",
        ],
        "metadata": {"environment": "development", "version": "1.0.0", "query_type": "storage_recommendation"},
    },
)

try:
    # Run the agent and get the response within the MCP client context
    response = agent(query)
except Exception as e:
    print(f"\nError: {e}")

### Now go to your Lanfuse console again, you should be able to see a new trace with the name "strands-agent-custom-tool-example" appear. Have a look at what's in the trace.

## Strands Agents with MCP servers
The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that standardizes how applications provide context to Large Language Models (LLMs). Strands Agents integrates with MCP to extend agent capabilities through external tools and services.

MCP enables communication between agents and MCP servers that provide additional tools. Strands includes built-in support for connecting to MCP servers and using their tools.

When working with MCP tools in Strands, all agent operations must be performed within the MCP client's context manager (using a with statement). This requirement ensures that the MCP session remains active and connected while the agent is using the tools. If you attempt to use an agent or its MCP tools outside of this context, you'll encounter errors because the MCP session will have closed.

#### Initialize MCP client for AWS documentation and AWS Pricing

In [ ]:
aws_docs_mcp = MCPClient(
    lambda: stdio_client(StdioServerParameters(command="uvx", args=["awslabs.aws-documentation-mcp-server@latest"]))
)

# Initialize MCP client for AWS pricing
aws_pricing_mcp = MCPClient(
    lambda: stdio_client(StdioServerParameters(command="uvx", args=["awslabs.aws-pricing-mcp-server@latest"]))
)

#### Test query that requires both AWS documentation and cost estimation

In [ ]:
query = """
I'm planning to use S3 for storing 1TB of data.
1. What storage class would you recommend for data that is accessed infrequently based on the aws document?
2. Can you estimate the monthly cost for storing 1TB in this storage class?
"""

print(f"\n=== QUERY: {query} ===\n")
print("Processing query...")

# IMPORTANT: Use the MCP client within a context manager
with aws_docs_mcp, aws_pricing_mcp:
    # Get the available tools from the MCP server
    mcp_tools = aws_docs_mcp.list_tools_sync() + aws_pricing_mcp.list_tools_sync()
    print(f"Available MCP tools: {len(mcp_tools)} tools found")

    # Create the agent with our custom tools and MCP tools
    agent = Agent(
        system_prompt="""You are an AWS assistant specialized in helping users with AWS-related tasks.
        You can provide information about AWS services, estimate costs, and help manage AWS resources.

        When answering questions:
        1. Use the AWS documentation when needed to provide accurate information
        2. Provide clear explanations with examples
        3. Consider best practices and security recommendations
        4. Be precise and accurate in your responses

        For resource tagging, use the tag_aws_resources tool with the resource ID and tag key-value pairs.
        For AWS documentation queries, use the AWS documentation MCP tools.
        For cost estimation, use the AWS Pricing MCP tools.
        """,
        tools=[
            # MCP tools
            *mcp_tools
        ],
        model="global.anthropic.claude-sonnet-4-5-20250929-v1:0",  # Using Claude 4.5 Sonnet via Bedrock
        name="strands-agent-mcp_example",
        trace_attributes={
            "session.id": "aws-mcp-agent-demo-session",
            "user.id": "example-user@example.com",
            "langfuse.tags": ["AWS-MCP-Agent", "MCP"],
            "metadata": {"environment": "development", "version": "1.0.0", "query_type": "storage_recommendation"},
        },
    )

    try:
        # Run the agent and get the response within the MCP client context
        response = agent(query)
        # print("\nAgent messages:")
        # print(json.dumps(agent.messages, indent=4))
    except Exception as e:
        print(f"\nError: {e}")

#### Now you know the drill, go to your Lanfuse console, you should be able to see a new trace with the name "strands-agent-mc-example" appear. Have a look at what's in the trace.

#### You can also use the built in tracing to show what the agent traces locally

In [ ]:
print("\nAgent messages:")
print(json.dumps(agent.messages, indent=4))

You can go to the Langfuse console to see the details of the traces.

## Lab4 Summary:

In Lab 4, we explored advanced observability for **Strands Agents** using `Langfuse`, demonstrating three distinct
tool integration approaches with comprehensive tracing capabilities. 

We successfully implemented built-in tools using the `image_reader` for AWS architecture analysis, created custom tools like the 
`aws_health_status_checker` for real-time service monitoring, and integrated `MCP (Model Context Protocol)` tools combining AWS Documentation and Pricing servers for complex multi-tool workflows. The key achievement was establishing complete observability through OpenTelemetry integration with StrandsTelemetry, enabling real-time monitoring of agent decision-making processes, tool usage patterns, and performance metrics in the Langfuse console. 

As we conclude this lab, you've gained valuable experience in building transparent, monitorable AI agents that transform from black boxes into production-ready systems with comprehensive observability, preparing you for deploying robust agentic systems with confidence in enterprise environments.